In [1]:
# 단일 배치 기반으로 시퀀스 분류기 학습 방법
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

checkpoint = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = ["I've been waiting for a HuggingFace course my whole life.",
             "This course is amazing!"]
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors='pt')
batch['labels'] = torch.tensor([1, 1])

optimizer = AdamW(model.parameters())
loss = model(**batch).loss
loss.backward()
optimizer.step()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [2]:
# 허브에서 데이터셋 로딩
from datasets import load_dataset

raw_datasets = load_dataset('glue', 'mrpc')
raw_datasets

Found cached dataset glue (/root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [3]:
raw_train_dataset = raw_datasets['train']
print(raw_train_dataset[0])

# 어떤 정수가 어떤 레이블인지 확인할 때는 features 값 확인
print(raw_train_dataset.features)

# label은 ClassLabel 타입이고 레이블 이름에 대한 정수 매핑은 names 폴더에 저장되어 있음
# 0은 not_equivalent, 1은 equivalent

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .', 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .', 'label': 1, 'idx': 0}
{'sentence1': Value(dtype='string', id=None), 'sentence2': Value(dtype='string', id=None), 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None), 'idx': Value(dtype='int32', id=None)}


데이터셋 전처리  
- 텍스트를 모델이 이해할 수 있는 숫자로 변환해야 함
- 토크나이저에 단일 문장 또는 다중 문장 리스트를 입력할 수 있으므로 다음과 같이 각 쌍의 모든 첫 번째 문장과 두 번째 문장을 각각 직접 토큰화할 수 있음

In [4]:
from transformers import AutoTokenizer

checkpoint = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_sentences_1 = tokenizer(raw_datasets['train']['sentence1'])
tokenized_sentences_2 = tokenizer(raw_datasets['train']['sentence2'])

- 두 개의 시퀀스를 모델에 전달하여 두 문장이 의역인지 아닌지에 대한 예측을 바로 얻을 수 없음
- 두 시퀀스를 쌍(pair)으로 처리(단일 매개변수로 처리)하고 적절한 전처리를 적용해야 함
- tokenizer는 한 쌍의 시퀀스를 가져와 BERT 모델이 요구하는 입력 형태로 구성할 수 있음

In [5]:
inputs = tokenizer('This is the first sentence.', 'This is the second one.')
print(inputs)

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


- 위 코드에서 만일 첫번째, 두번째 매개변수가 다중 문자열이 담긴 리스트라면 각 리스트에 저장된 문자열(문장)의 순서대로 한 쌍씩 토큰화됨
- 첫번째 리스트의 첫번째 문자열과 두번째 리스트의 첫번째 문자열이 하나의 문자열 쌍으로 입력되는 방식
- token_type_ids는 전체 입력(input_ids)의 어느 부분이 첫 번째 문장이고 어느 것이 두 번째 문장인지 모델에 알려줌

In [6]:
# input_ids 내부의 ID들을 다시 단어로 디코딩하면 아래와 같음
# 모델 입장에서 입력값이 "[CLS] 문장1 [SEP] 문장2 [SEP]"와 같이 될 것으로 알 수 있음
print(tokenizer.convert_ids_to_tokens(inputs['input_ids']))

['[CLS]', 'this', 'is', 'the', 'first', 'sentence', '.', '[SEP]', 'this', 'is', 'the', 'second', 'one', '.', '[SEP]']


- "[CLS] 문장1 [SEP]"에 부분은 token_type_id가 0, "문장2 [SEP]"에 해당하는 부분은 1임
- 다른 checkpoint를 선택한다면 토큰화된 입력(tokenized inputs)에 token_type_ids가 존재하지 않을 수도 있음
  - DistilBERT 모델을 사용하는 경우에는 tokenizer가 token_type_ids를 반환하지 않음
  - 모델이 사전학습 과정에서 이러한 형태의 입력 형식으로 학습을 진행했을 경우에만 반환됨
- 사전 학습 과정에서 다음 문장 예측(next sentence prediction)을 사용하면 모델에 무작위로 마스킹된 토큰(masked tokens)이 포함된 문장 쌍이 입력되고 두 번째 문장이 첫 번째 문장을 따르는지 여부를 예측하도록 요구됨
- 학습 과정에서 next sentence prediction을 어렵지 않게 하기 위해 입력의 약 50%는 두 문장이 원본 문서에서 연속적으로 나타나는 쌍 집합이며 나머지 50%는 문장 쌍을 서로 다른 문서에서 추출된 문장들로 구성함
- 토큰화 완료된 입력에 token_type_ids가 있는지 여부에 대해 걱정할 필요가 없음
  - 토크나이저와 모델 모두에 동일한 checkpoint를 사용하면 토크나이저는 모델에 무엇을 제공해야 하는지 알고 있기 때문에 문제가 없음

In [7]:
# 학습 데이터셋을 전처리하는 하나의 방법
tokenized_dataset = tokenizer(
    raw_datasets['train']['sentence1'], raw_datasets['train']['sentence2'],
    padding=True, truncation=True)

In [8]:
# - 특정 데이터를 dataset 객체로 유지하기 위해 Dataset.map() 메서드를 사용함
def tokenize_function(example):
    return tokenizer(example['sentence1'], example['sentence2'], truncation=True)

- 위 함수는 데이터셋의 개별 항목이 담겨진 딕셔너리를 입력받아서 input_ids, attention_mask 및 token_type_ids 키가 지정된 딕셔너리를 반환함 
- tokenizer는 문장 쌍 리스트에서 작동하기 때문에 example에 여러 샘플이 포함된 경우에도 작동함 
  - map()에서 batched=True 옵션을 사용할 수 있어 토큰화 속도가 빨라짐
- 모든 샘플들을 최대 길이로 채우는 것이 효율적이지 않기 때문에 배치(batch) 형태로 실행할 때 채움
  - 전체 데이터셋에서의 최대 길이가 아니라 해당 batch 내에서의 최대 길이로 채우기만 하면 되기 때문
  - 입력의 길이가 매우 가변적일 때 시간과 처리 능력 절약 가능

In [9]:
# 한 번에 모든 데이터셋에 토큰화를 적용하는 방법
# map메서드에서 batched=True를 사용하면 각 batch의 모든 요소들에 한꺼번에 적용됨
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-3dde15d73cdbba3e.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-aeb1355e0d6fe019.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-4b5667fe9e3cdb60.arrow


DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

- Datasets 라이브러리는 datasets에 새로운 필드들을 추가함
- 이 필드들은 전처리 함수에서 반환된 사전의 각 키(input_ids, token_type_ids, attention_mask)값
- num_proc 매개변수를 전달하여 map()으로 전처리 기능을 적용할 때 multi-processing을 사용할 수 있음
- 위의 tokenize_function은 input_ids, attention_mask, token_type_ids 키가 존재하는 딕셔너리를 반환하므로 이 3개의 새로운 필드가 데이터셋의 모든 분할(학습, 검증, 평가)에 추가됨
- 전처리 함수가 map()을 적용한 데이터셋의 기존 idx, label 키 등에 대한 새로운 값을 반환한 경우 기존 필드(idx, label, sentence1, sentence2 등)를 변경할 수 있음

동적 패딩(Dynamic padding)  
- 샘플을 함께 모아 지정된 크기의 batch로 구성하는 역할을 하는 함수를 collate function이라고 함
- 이 함수는 DataLoader를 build할 때 전달할 수 있는 매개변수
  - 기본값은 샘플들을 PyTorch 텐서로 변환하고 결합하는 함수
  - 만일 대상 샘플들이 리스트, 튜플, 딕셔너리면 재귀적으로 이 작업이 수행됨
  - 예제의 경우 입력값이 모두 동일한 길이가 아니기 때문에 이 작업이 불가능함
- 전체 데이터셋이 아닌 개별 batch에 대해 별도 padding을 수행하여 과도하게 긴 입력으로 인한 과도한 padding 작업을 방지함
- 실제로 이를 수행하려면 batch로 분리하려는 데이터셋의 요소에 대해서 정확한 수의 패딩(padding)을 적용할 수 있는 collate function을 정의해야 함
  - Transformers 라이브러리의 DataCollatorWithPadding을 통해 됨
  - 이 함수는 토크나이저를 입력으로 받음. 사용하려는 패딩 토큰(padding token)이 무엇인지와 모델이 입력의 왼쪽 혹은 오른쯕 중 어느 쪽에 패딩(padding)을 수행할지를 파악하기 위함

In [10]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 학습데이터에서 batch로 묶을 몇개의 샘플로 테스트
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}

# 32에서 67까지 다양한 길이의 샘플을 얻을 수 있음
# 동적 패딩은 이 batch 내의 모든 샘플들이 배치 내부에서 최대 길이인 67 길이로 
# padding 되어야 함. dynamic padding이 없으면 모든 샘플들은 전체 데이터셋의 최대 
# 길이 또는 모델이 허용할 수 있는 최대 길이로 채워져야 함
print([len(x) for x in samples["input_ids"]])

[50, 59, 47, 67, 59, 50, 62, 32]


In [11]:
# 동적으로 batch를 padding 하는지 확인
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': torch.Size([8, 67]),
 'token_type_ids': torch.Size([8, 67]),
 'attention_mask': torch.Size([8, 67]),
 'labels': torch.Size([8])}

Trainer API를 이용한 모델 미세 조정(fine-tuning)  

In [12]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset('glue', 'mrpc')
checkpoint = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(
        example['sentence1'], example['sentence2'], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Found cached dataset glue (/root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-e25e0335d4cc5aef.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-01b1d4497b862966.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-c9ea211c9fd30c10.arrow


학습 (Training)  
- 먼저 Trainer가 학습 및 평가에 사용할 하이퍼파라미터(hyperparameters)를 포함하는 TrainingArguments 클래스를 정의해야 함
- 여기서 제공해야 할 매개변수는 학습된 모델이 저장될 디렉토리 값
  - 나머지는 모두 기본값(default values)을 활용해도 됨

In [13]:
from transformers import TrainingArguments
training_args = TrainingArguments('test-trainer')

# 두 개의 레이블이 있는 AutoModelForSequenceClassification 클래스 사용
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [14]:
# Trainer 정의
from transformers import Trainer
trainer = Trainer(
    model, training_args, 
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=data_collator, # 이전에 정의된 DataCollatorWithPadding
    tokenizer=tokenizer,)

trainer.train()

/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
500,0.349600


TrainOutput(global_step=690, training_loss=0.2794818352961886, metrics={'train_runtime': 116.0851, 'train_samples_per_second': 94.793, 'train_steps_per_second': 5.944, 'total_flos': 430291408824720.0, 'train_loss': 0.2794818352961886, 'epoch': 3.0})

- 500단계마다 학습 손실(training loss)이 보고되지만 모델의 성능이 좋은지 혹은 나쁜지는 알려주지 않음
  - 학습 과정에서 evaluation_strategy를 "steps"(매 eval_steps마다 평가)나 "epoch"(각 epoch 마지막에 평가) 등으로 지정하지 않았음
  - 평가 방법 혹은 평가 척도를 정의한 compute_metrics() 함수를 Trainer에 지정하지 않았음

평가 (Evaluation)  
- compute_metrics()를 구현하고 학습할 때 사용하는 방법
  - 이 함수는 EvalPrediction 객체(predictions 필드와 label_ids 필드가 포함된 네임드튜플(named tuple))를 필요로 하며 문자열과 실수값(floats)을 매핑하는 딕셔너리를 반환함
  - 여기서 문자열은 반환된 메트릭(metrics)의 이름이고 실수값(floats)은 해당 메트릭에 기반한 평가 결과값

In [15]:
# 모델에서 예측 결과를 얻을 때 쓰는 method
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(408, 2) (408,)


- predict()는 predictions, label_ids, metrics가 있는 네임드튜플(named tuple)값
- metrics 필드에는 전달된 데이터셋의 loss와 시간 메트릭(time metrics) 값만 포함됨
- 시간 메트릭(time metrics)은 예측에 걸린 전체 및 평균 시간
- compute_metrics() 함수를 구성하고 Trainer에 전달하면 해당 필드에는 compute_metrics()에서 반환한 메트릭(metrics)도 포함됨
- predictions은 408 x 2인 2차원 배열
  - 408 개는 우리가 예측에 사용한 데이터셋의 개수
  - 해당 값은 predict()에 전달한 데이터셋의 각 요소에 대한 로짓(logit)값
  - 모든 Transformer 모델은 로짓(logit)값을 반환함

In [16]:
# 이 로짓(logit)값들을 레이블과 비교할 수 있는 예측 결과로 변환하려면 
# 두 번째 축에 존재하는 항목에서 최대값이 있는 인덱스를 가져와야 함
import numpy as np
preds = np.argmax(predictions.predictions, axis=-1)

In [17]:
# compute_metric() 함수를 구성하기 위해 Datasets 라이브러리 사용
# load_metric() 함수를 사용하여 MRPC 데이터셋과 관련된 metric을 로드함
# 로드된 객체에는 메트릭(metrics) 계산을 사용할 수 있는 compute() 메서드가 있음
from datasets import load_metric

metric = load_metric("glue", "mrpc")
# 모델 헤드를 무작위로 초기화하면 계산된 메트릭이 변경될 수 있어 정확한 결과는 다를 수 있음
metric.compute(predictions=preds, references=predictions.label_ids)

/tmp/ipykernel_25530/1271866207.py:6: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue", "mrpc")


{'accuracy': 0.8799019607843137, 'f1': 0.9144851657940662}

In [18]:
# 평가 함수 정리
def compute_metrics(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [19]:
# 각 epoch가 끝날 때 메트릭(metrics)을 출력하도록 하기 위해 
# compute_metrics() 함수를 사용하여 새 Trainer를 정의하는 방법
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
trainer = Trainer(
    model, training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# evaluation_strategy 값을 "epoch"으로 설정하고 새로운 TrainingArguments 및 모델 생성
trainer.train()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.421852,0.835784,0.889984
2,No log,0.375808,0.845588,0.893761


TrainOutput(global_step=690, training_loss=0.32166418545487996, metrics={'train_runtime': 121.0357, 'train_samples_per_second': 90.915, 'train_steps_per_second': 5.701, 'total_flos': 430291408824720.0, 'train_loss': 0.32166418545487996, 'epoch': 3.0})

전체 학습 (Full Training)

In [20]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Found cached dataset glue (/root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-cb0e5869fff58506.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-3f2266ca56c4f876.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-3d7bd5692b533052.arrow


학습을 위한 준비  
- 학습 루프(training loop)를 작성하기 전에 몇 가지 객체를 정의해야 함
  - 배치(batch)를 반복하는 데 사용할 dataloaders 객체
  - 이 dataloaders를 정의하기 전에 tokenized_datasets에 후처리를 적용해야 함
    - 모델이 필요로 하지 않는 값이 저장된 columns 제거
    - column label의 이름을 labels로 바꿔야 함. 이는 모델이 labels라는 이름으로 매개변수를 받기 때문.
    - 파이썬 리스트 대신 PyTorch 텐서(tensors)를 반환하도록 datasets의 형식을 설정 해야 함

In [21]:
# tokenized_datasets에 이런 작업을 위한 별도의 메서드 존재
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [22]:
# 위 코드로 인해 결과적으로 tokenized_datasets에는 
# 모델이 허용하는 columns만 존재함을 알 수 있음
from torch.utils.data import DataLoader
train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator,)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator,)

In [23]:
# 데이터 처리에 오류 확인을 위해 batch 체크
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

# 실제 데이터 shape 이 다를 수 있는데 이는 학습 dataloader에 대해 shuffle=True를 
# 설정하고 배치(batch) 내에서의 최대 길이로 패딩(padding)하기 때문

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 68]),
 'token_type_ids': torch.Size([8, 68]),
 'attention_mask': torch.Size([8, 68])}

In [24]:
# 모델 인스턴스화
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# batch 전달
outputs = model(**batch)

# 모든 Transformers 모델은 매개변수에 labels이 포함되어 있다면 손실(loss)과 함께 
# logit값도 반환함 (batch 내의 logit값이 2개이므로 8 x 2인 텐서)
print(outputs.loss, outputs.logits.shape)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

tensor(0.6796, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


In [25]:
# 최적화 함수(optimizer) 지정
# weight decay regularization 적용
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)

/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


마지막으로, Trainer에서 디폴트로 사용되는 학습률 스케줄러(learning rate scheduler)는 최대값(5e-5)에서 0까지 선형 감쇠(linear decay)합니다. 이를 적절하게 정의하려면 우리가 수행할 학습 단계의 횟수를 알아야 합니다. 이는 실행하려는 에포크(epochs) 수에 학습 배치(batch)의 개수를 곱한 것입니다. 학습 배치의 개수는 학습 dataloader의 길이와 같습니다. Trainer는 디폴트로 3개의 에포크(epochs)를 사용하므로 다음을 따릅니다:

In [26]:
# 학습률 스케줄러(learning rate scheduler)
# 최대값(5e-5)에서 0까지 선형 감쇠(linear decay)함. 
# 이를 적절하게 정의하려면 수행할 학습 단계의 횟수를 알아야 함.
# -> 실행하려는 에포크(epochs) 수에 학습 배치(batch)의 개수를 곱한 것
# 학습 배치 개수 = train dataloader length
from transformers import get_scheduler
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=0,
    num_training_steps=num_training_steps,)
print(num_training_steps)

1377


학습 루프 (Training Loop)

In [27]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [28]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/1377 [00:00<?, ?it/s]

평가 루프 (Evaluation Loop)

In [29]:
from datasets import load_metric

metric = load_metric("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    # metric.add_batch() 메서드로 평가 루프(evaluation loop)를 실행하면서 
    # 배치(batch)별 평가 메트릭(metrics) 계산 결과를 누적할 수 있음
    metric.add_batch(predictions=predictions, references=batch["labels"])

# 모든 배치(batch)를 누적하고 나면 metric.compute()로 최종 결과를 얻을 수 있음
metric.compute()

{'accuracy': 0.8480392156862745, 'f1': 0.8931034482758621}

Accelerate를 사용한 학습 루프 가속화  
- Accelerate 라이브러리를 사용하여 여러 GPU 또는 TPU에서 분산 학습(distributed training)을 수행할 수 있음

In [31]:
from accelerate import Accelerator
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

# 분산 설정을 초기화하는 Accelerator 객체 초기화
# 장치 배치(device placement)를 자동으로 처리하므로 장치에 모델을 배치하는 
# model.to(device)을 제거할 수 있음.
# 필요한 경우 device 대신 accelerator.device를 사용할 수 있음
accelerator = Accelerator()

# 제거
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# model.to(device)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5)

# model 및 optimizer를 입력함
# 입력한 객체들을 적절한 컨테이너로 감싸서 분산 학습이 의도대로 작동되도록 함
train_dataloader, eval_dataloader, model, optimizer = accelerator.prepare(
    train_dataloader, eval_dataloader, model, optimizer)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=0, 
    num_training_steps=num_training_steps)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        # batch = {k: v.to(device) for k, v in batch.items()} 제거
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss) # loss.backward 대체

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

  0%|          | 0/1377 [00:00<?, ?it/s]

전체 코드 정리

In [6]:
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, AdamW, get_scheduler

import torch
from torch.utils.data import DataLoader

from tqdm.auto import tqdm

# 데이터 셋 적재
raw_datasets = load_dataset("glue", "mrpc")
# 사전학습 언어모델 checkpoint 이름 지정
checkpoint = "bert-base-uncased"
# 지정된 사전학습 언어모델에서 토크나이저 인스턴스화
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# 토크나이저 함수 사용자 정의화 (sentence1, sentence2 컬럼에 대해서만 토크나이징 수행)
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

# 토크나이징 수행
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
# 배치(batch)별 패딩(padding)을 위한 data collator 정의
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 불필요한 입력 컬럼을 제거하고 사전학습 언어모델에 필요한 입력만 남김.
tokenized_datasets = tokenized_datasets.remove_columns(
    ["sentence1", "sentence2", "idx"])
# 데이터셋의 label 컬럼명을 labels로 변경
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
# 데이터셋의 유형을 PyTorch tensor로 변경
tokenized_datasets.set_format("torch")

# 변경된 컬럼 출력
print(tokenized_datasets["train"].column_names)


# 각 종류별 데이터 로더 생성
train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], shuffle=True, batch_size=8,
    collate_fn=data_collator)

# 사전학습 언어모델 인스턴스화
model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, num_labels=2)
# 최적화 함수 정의
optimizer = AdamW(model.parameters(), lr=5e-5)

# 에포크 개수 설정
num_epochs = 3
# 학습 스텝 수 계산
num_training_steps = num_epochs * len(train_dataloader)
# 학습 스케쥴러 설정
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=0, 
    num_training_steps=num_training_steps)

# GPU로 모델을 이동
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# 진행 상황바 정의
progress_bar = tqdm(range(num_training_steps))

# 모델을 학습 모드로 전환
model.train()
# 학습 루프 시작
for epoch in range(num_epochs):
    for batch in train_dataloader:
        # 현재 배치 중에서 입력값을 모두 GPU로 이동.
        batch = {k: v.to(device) for k, v in batch.items()}
        # 모델 실행
        outputs = model(**batch)
        # 손실값 가져오기
        loss = outputs.loss
        # 역전파 수행
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

# 평가 메트릭 가져오기
metric = load_metric("glue", "mrpc")
# 모델을 평가 모드로 전환
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

# 평가 결과 계산 및 출력 
metric.compute()

Found cached dataset glue (/root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-cb0e5869fff58506.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-3f2266ca56c4f876.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-3d7bd5692b533052.arrow


['labels', 'input_ids', 'token_type_ids', 'attention_mask']


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

  0%|          | 0/1377 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/tmp/ipykernel_29121/115193445.py:88: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue", "mrpc")


{'accuracy': 0.8480392156862745, 'f1': 0.8916083916083916}